# Amplifierd API - Mount Plan Generation

This notebook demonstrates mount plan generation from cached profiles.

## Overview

Mount plans are complete specifications for session initialization. They contain:
- **Session configuration**: Profile, settings, parent session
- **EmbeddedMounts**: Agents and context with content embedded (for LLM consumption)
- **ReferencedMounts**: Providers, tools, hooks with file:// paths (for runtime loading)
- **Organization**: Automatic grouping by module type (agents, context, providers, tools, hooks)

**Key Concepts**:
- **Module IDs**: Unique identifiers following pattern `{profile}.{type}.{name}`
- **Two mount types**: Embedded (text content) vs Referenced (file paths)
- **Format versioning**: Plans include `formatVersion` for compatibility
- **Organized views**: Mount points auto-organized into dictionaries by type

This notebook covers mount plan generation and inspection.

## Setup

Import required libraries and set up helper functions.

In [ ]:
import json
from typing import Any

import requests

BASE_URL = "http://127.0.0.1:8421"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> Any | None:
    """Print formatted API response."""
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

✓ Setup complete


## Generate Mount Plan

Create a mount plan from a cached profile. This converts the profile's resources into a structured mount plan ready for session initialization.

The mount plan includes:
- Session configuration (profile ID, session ID, settings)
- All mount points organized by type
- Embedded content for agents and context
- File references for providers, tools, and hooks

In [ ]:
# Generate mount plan for developer-expertise/dev profile
request_data = {
    "profile_id": "developer-expertise/dev",
    "amplified_dir": ".",  # Use "." for root amplified directory
    "settings_overrides": {"llm": {"model": "claude-3-5-sonnet-20241022"}},
}

response = requests.post(f"{API_BASE}/mount-plans/generate", json=request_data)
mount_plan = print_response(response, "GENERATE MOUNT PLAN")

if mount_plan:
    print("\n✓ Mount plan generated")
    print(f"  Session ID: {mount_plan['session']['sessionId']}")
    print(f"  Profile ID: {mount_plan['session']['profileId']}")
    print(f"  Format Version: {mount_plan['formatVersion']}")
    print(f"  Total Mount Points: {len(mount_plan['mountPoints'])}")

## Inspect Mount Plan Structure

Examine the organized views and mount point types.

The mount plan provides organized dictionaries for each module type:
- `agents`: Embedded agent instructions
- `context`: Embedded context documents
- `providers`: Referenced provider modules (file:// paths)
- `tools`: Referenced tool modules (file:// paths)
- `hooks`: Referenced hook modules (file:// paths)

In [ ]:
if mount_plan:
    print("Mount Plan Organization:")
    print(f"  Agents: {len(mount_plan['agents'])} embedded mounts")
    print(f"  Context: {len(mount_plan['context'])} embedded mounts")
    print(f"  Providers: {len(mount_plan['providers'])} referenced mounts")
    print(f"  Tools: {len(mount_plan['tools'])} referenced mounts")
    print(f"  Hooks: {len(mount_plan['hooks'])} referenced mounts")

    print("\n✓ All resources organized by type")

## Examine Mount Points

Inspect the structure of embedded vs referenced mounts.

### Embedded Mounts (Agents, Context)
- Content included directly in mount plan
- Ready for LLM consumption
- Used for system instructions and context documents

### Referenced Mounts (Providers, Tools, Hooks)
- File paths (file:// URLs) for runtime loading
- Used for executable code modules
- Loaded dynamically by amplifier-core

In [ ]:
if mount_plan:
    # Examine embedded mount (agent)
    if mount_plan["agents"]:
        print("Sample Embedded Mount (Agent):")
        first_agent_key = list(mount_plan["agents"].keys())[0]
        agent = mount_plan["agents"][first_agent_key]
        print(f"  Module ID: {agent['moduleId']}")
        print(f"  Mount Type: {agent['mountType']}")
        print(f"  Module Type: {agent['moduleType']}")
        print(f"  Content Length: {len(agent['content'])} characters")
        print(f"  Content Preview: {agent['content'][:100]}...")

    # Examine referenced mount (provider)
    if mount_plan["providers"]:
        print("\nSample Referenced Mount (Provider):")
        first_provider_key = list(mount_plan["providers"].keys())[0]
        provider = mount_plan["providers"][first_provider_key]
        print(f"  Module ID: {provider['moduleId']}")
        print(f"  Mount Type: {provider['mountType']}")
        print(f"  Module Type: {provider['moduleType']}")
        print(f"  Source Path: {provider['sourcePath']}")

    print("\n✓ Mount point structure verified")

## List All Module IDs

Show all module IDs in the mount plan, demonstrating the `{profile}.{type}.{name}` convention.

In [ ]:
if mount_plan:
    print("All Module IDs in Mount Plan:")
    print("\nAgents:")
    for key in sorted(mount_plan["agents"].keys()):
        print(f"  - {key}")

    print("\nContext:")
    for key in sorted(mount_plan["context"].keys()):
        print(f"  - {key}")

    print("\nProviders:")
    for key in sorted(mount_plan["providers"].keys()):
        print(f"  - {key}")

    print("\nTools:")
    for key in sorted(mount_plan["tools"].keys()):
        print(f"  - {key}")

    print("\nHooks:")
    for key in sorted(mount_plan["hooks"].keys()):
        print(f"  - {key}")

    print("\n✓ Module ID convention: {profile}.{type}.{name}")

## Generate with Custom Session ID

Generate a mount plan with a custom session ID instead of using auto-generated UUIDs.

Custom session IDs are useful for:
- Testing and development
- Predictable session identifiers
- Integration with external systems

In [ ]:
# Generate with custom session ID
custom_request = {
    "profile_id": "foundation/base",
    "amplified_dir": ".",  # Use "." for root amplified directory
    "session_id": "my-custom-session-123",
    "settings_overrides": {"session": {"maxTurns": 50}},
}

response = requests.post(f"{API_BASE}/mount-plans/generate", json=custom_request)
custom_plan = print_response(response, "MOUNT PLAN WITH CUSTOM SESSION ID")

if custom_plan:
    print(f"\n✓ Custom session ID preserved: {custom_plan['session']['sessionId']}")
    print(f"  Settings override applied: maxTurns = {custom_plan['session']['settings']['session']['maxTurns']}")

## Compare Different Profiles

Generate mount plans for different profiles to see how they differ in structure and content.

In [ ]:
# Compare foundation/base vs developer-expertise/dev
profiles = ["foundation/base", "developer-expertise/dev"]
profile_stats = {}

for profile_id in profiles:
    response = requests.post(
        f"{API_BASE}/mount-plans/generate",
        json={"profile_id": profile_id, "amplified_dir": "."}  # Updated to snake_case with amplified_dir
    )
    if response.status_code == 201:  # mount plan generation returns 201 Created
        plan = response.json()
        profile_stats[profile_id] = {
            "agents": len(plan["agents"]),
            "context": len(plan["context"]),
            "providers": len(plan["providers"]),
            "tools": len(plan["tools"]),
            "hooks": len(plan["hooks"]),
            "total": len(plan["mountPoints"]),
        }
    else:
        print(f"Warning: Failed to generate plan for {profile_id}: {response.status_code}")

print("Profile Comparison:")
print("\n" + "-" * 80)
print(f"{'Profile':<30} {'Agents':<10} {'Context':<10} {'Providers':<12} {'Tools':<10} {'Hooks':<10} {'Total':<10}")
print("-" * 80)
for profile_id, stats in profile_stats.items():
    print(
        f"{profile_id:<30} {stats['agents']:<10} {stats['context']:<10} {stats['providers']:<12} {stats['tools']:<10} {stats['hooks']:<10} {stats['total']:<10}"
    )
print("-" * 80)

print(f"\n✓ Profile comparison complete ({len(profile_stats)}/{len(profiles)} profiles)")

## Inspect Session Settings

Examine the session configuration and settings in the generated mount plan.

In [ ]:
if mount_plan:
    session_config = mount_plan["session"]
    print("Session Configuration:")
    print(f"  Session ID: {session_config['sessionId']}")
    print(f"  Profile ID: {session_config['profileId']}")
    print(f"  Parent Session: {session_config.get('parentSessionId', 'None')}")

    print("\nSettings:")
    settings = session_config["settings"]

    # Show actual settings (don't assume specific keys)
    if settings:
        for category, config in settings.items():
            print(f"  {category.upper()}:")
            if isinstance(config, dict):
                for key, value in config.items():
                    print(f"    {key}: {value}")
            else:
                print(f"    {config}")
    else:
        print("  (No settings overrides)")

    print("\n✓ Session configuration verified")

## Error Handling

Test error scenarios to verify proper error handling.

In [ ]:
# Try to generate mount plan for non-existent profile
error_request = {
    "profile_id": "non-existent/profile",
    "amplified_dir": "."
}

response = requests.post(f"{API_BASE}/mount-plans/generate", json=error_request)
print_response(response, "ERROR: NON-EXISTENT PROFILE")

if response.status_code == 404:
    print("\n✓ Correctly returns 404 for non-existent profile")

In [ ]:
# Try to generate mount plan with invalid request (missing profile_id)
invalid_request = {
    "session_id": "test-session",
    "amplified_dir": "."
}

response = requests.post(f"{API_BASE}/mount-plans/generate", json=invalid_request)
print_response(response, "ERROR: MISSING PROFILE ID")

if response.status_code == 422:
    print("\n✓ Correctly returns 422 for validation error")

## Content Analysis

Analyze the content embedded in agents and context mounts.

In [ ]:
if mount_plan:
    print("Content Analysis:")

    # Analyze agent content
    print("\nAgent Content:")
    total_agent_chars = sum(len(agent["content"]) for agent in mount_plan["agents"].values())
    print(f"  Total agents: {len(mount_plan['agents'])}")
    print(f"  Total characters: {total_agent_chars:,}")
    print(f"  Average per agent: {total_agent_chars // max(len(mount_plan['agents']), 1):,} characters")

    # Analyze context content
    print("\nContext Content:")
    total_context_chars = sum(len(ctx["content"]) for ctx in mount_plan["context"].values())
    print(f"  Total context documents: {len(mount_plan['context'])}")
    print(f"  Total characters: {total_context_chars:,}")
    print(f"  Average per document: {total_context_chars // max(len(mount_plan['context']), 1):,} characters")

    # Total embedded content
    print("\nTotal Embedded Content:")
    total_chars = total_agent_chars + total_context_chars
    print(f"  Total characters: {total_chars:,}")
    print(f"  Approximate tokens (÷4): {total_chars // 4:,}")

    print("\n✓ Content analysis complete")

## Summary

### Mount Plan Generation
- ✓ Generate mount plans from cached profiles
- ✓ Automatic session ID generation (UUID-based)
- ✓ Custom session IDs supported
- ✓ Settings overrides applied
- ✓ Two mount types: embedded (agents/context) vs referenced (modules)
- ✓ Automatic organization by module type

### Mount Point Types

**EmbeddedMount** (agents, context):
- Content embedded as text in mount plan
- Ready for LLM consumption
- Used for system instructions and context

**ReferencedMount** (providers, tools, hooks):
- File paths (file:// URLs) for runtime loading
- Used for executable code modules
- Loaded dynamically by amplifier-core

### API Endpoints

| Method | Endpoint | Description |
|--------|----------|-------------|
| POST | `/api/v1/mount-plans/generate` | Generate mount plan from profile |

### Module ID Convention

Module IDs follow the pattern: `{profile}.{type}.{name}`

Examples:
- `developer-expertise/dev.agent.zen-architect`
- `developer-expertise/dev.context.IMPLEMENTATION_PHILOSOPHY`
- `developer-expertise/dev.provider.anthropic`

### Format Version

The mount plan includes a `formatVersion` field for compatibility tracking. This allows amplifier-core to handle different mount plan versions gracefully.

### Next Steps

Continue to:
- **07-session-lifecycle.ipynb** - Enhanced session management with mount plans
- **08-runtime-integration.ipynb** - Using mount plans with amplifier-core